In [1]:
!pip install tensorflow==2.15.0 numpy==1.26.4 pandas==2.2.2 scikit-learn==1.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 69.7 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 48.2 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 40.8 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.8/208.8 MB 41.5 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 87.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 75.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 48.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 76.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 36.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import os

In [14]:
df = pd.read_csv('sensor_data_2.csv', header=None)

# Mock Data Generation: 1000 rows, 13 columns (A-M)
# Column A (Index 0): Target (0-9)
# Columns B-M (Indices 1-12): 11 Sensor Features
num_samples = df.shape[0]
num_features = 12

# Create a DataFrame for clarity
# Define the names for the four sensors, each having X, Y, Z axes.
SENSORS = [
    'Accelerometer',
    'Gyroscope',
    'Rotation_Vector',
    'Magnetic_Field'
]

# Define the axes
AXES = ['X', 'Y', 'Z']

# Generate the 12 feature columns by combining the sensor and axis names
# Example: ['Accelerometer_X', 'Accelerometer_Y', 'Accelerometer_Z', ...]
feature_columns = [f'{sensor}_{axis}' for sensor in SENSORS for axis in AXES]

# Recreate the full list of columns, starting with the target variable ('A_Target')
df.columns = ['Target'] + feature_columns

print(df.head())
print(f"Total samples: {df.shape[0]}, Total columns: {df.shape[1]}")

# --- 2. Data Preprocessing ---

# Separate features (X) and target (Y)
# X are columns 2 through 12 (C to M)
X = df.iloc[:, 1:].values
# Y is column 0 (A)
y = df.iloc[:, 0].values

# Standardize features (highly recommended for neural networks)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert target labels to one-hot encoding (needed for multi-class classification)
y_encoded = keras.utils.to_categorical(y, num_classes=10) # 10 classes: 0 through 9

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y
)


   Target  Accelerometer_X  Accelerometer_Y  Accelerometer_Z  Gyroscope_X  \
0       9        -0.899103         8.285747         4.893920     0.116981   
1       8        -0.629911         7.975278         5.528018     0.097738   
2       5        -1.357329         6.623333         7.004390     0.025809   
3       1        -2.093720         6.971489         6.479166     0.122478   
4       6        -1.146162         7.526025         6.053243     0.049480   

   Gyroscope_Y  Gyroscope_Z  Rotation_Vector_X  Rotation_Vector_Y  \
0     0.021075    -0.074373           0.435502          -0.150417   
1     0.010996    -0.094531           0.437782           0.083928   
2    -0.022907    -0.012370           0.346829           0.042343   
3     0.186619    -0.080481           0.381101           0.038290   
4    -0.015424    -0.058643           0.405773           0.037086   

   Rotation_Vector_Z  Magnetic_Field_X  Magnetic_Field_Y  Magnetic_Field_Z  
0          -0.323174           1.21875       

In [17]:

model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(num_features,)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


print("\n--- Model Training Started ---")
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    verbose=0
)
print("Model Training Complete.")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")


--- Model Training Started ---
Model Training Complete.
Test Accuracy: 0.2625


In [18]:

# TFLite Converter requires a saved model
tf.saved_model.save(model, 'har_model_saved')

# Initialize the converter
converter = tf.lite.TFLiteConverter.from_saved_model('har_model_saved')

# OPTIMIZATION: Quantization for smaller size and faster inference on mobile
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS # Enable TensorFlow Lite ops.
]

# Perform the conversion
tflite_model = converter.convert()

# Save the TFLite model to a file
tflite_model_path = Path('har_model.tflite')
tflite_model_path.write_bytes(tflite_model)
print(f"\nSuccessfully converted model to TFLite format and saved to: {tflite_model_path.resolve()}")


INFO:tensorflow:Assets written to: har_model_saved/assets


INFO:tensorflow:Assets written to: har_model_saved/assets



Successfully converted model to TFLite format and saved to: /Users/hmanasi1/AndroidStudioProjects/iot-project/models/har_model.tflite


2025-11-28 10:30:06.756064: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-11-28 10:30:06.756077: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-11-28 10:30:06.756280: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: har_model_saved
2025-11-28 10:30:06.756775: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-11-28 10:30:06.756778: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: har_model_saved
2025-11-28 10:30:06.757721: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-11-28 10:30:06.758090: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-11-28 10:30:06.773877: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: har_model_saved
2025-11-28